In [1]:
import vincent
from vincent import *
import pandas as pd
vincent.core.initialize_notebook()

In [2]:
state_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_states.topo.json"
county_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_counties.topo.json"
world_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/world-countries.topo.json"

In [3]:
geo_data = [{'name': 'counties',
             'url': county_topo,
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}
             ]

vis = Map(geo_data=geo_data, scale=1000, projection='albersUsa')
del vis.marks[1].properties.update
vis.marks[0].properties.update.fill.value = '#084081'
vis.marks[1].properties.enter.stroke.value = '#fff'
vis.marks[0].properties.enter.stroke.value = '#7bccc4'
vis.display()

In [4]:
import json
import pandas as pd

geo_data = [{'name': 'counties',
             'url': county_topo,
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}
             ]


#geo_data = [{'name': 'counties',
#             'url': county_topo,
#             'feature': 'us_counties.geo'}]

#Map the county codes we have in our geometry to those in the county_data file, 
with open('/Users/roellk/census_shape_files/counties/us_counties.topo.json', 'r') as f:
    get_id = json.load(f)

with open('us_counties.topo.json', 'w') as f:
    json.dump(get_id, f)
    
#Read into Dataframe, cast to string for consistency
df = pd.read_csv("../data/county_counts.txt", sep="|", na_values=[' '], dtype=object)
df.columns = ['FIPS', 'lar_count']
df['FIPS_Code'] = df['FIPS'].astype(str)

with open('us_counties.topo.json', 'w') as f:
    json.dump(get_id, f)
    
#Grab the FIPS codes from the topo json file and load them into a dataframe
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)


merged = pd.merge(df, county_df, on='FIPS', how='outer')
merged.fillna(0, inplace=True)
merged.lar_count = merged.lar_count.apply(lambda x: int(x))


vis = Map(data=merged, geo_data=geo_data, scale=1100, projection='albersUsa',
          data_bind='lar_count', data_key='FIPS',
          map_key={'counties': 'properties.FIPS'})

vis.marks[0].properties.enter.stroke_opacity = ValueRef(value=0.5)
vis.rebind(column='lar_count', brew='YlGnBu')
#vis.to_json('vega.json')
del vis.marks[1].properties.update
vis.display()

In [5]:
#create function that accepts data and map level (state/county) to produce a map
def draw_map(geo_level="county", data=None, local_path='/Users/roellk/census_shape_files/'):
    """Accepts geo level and data arguments and draws a Vega style map using the Vincent library."""
    #set topo urls for Vincent library
    state_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_states.topo.json"
    county_topo = "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_counties.topo.json"
    
    #set geo level
    if geo_level == "county":
        geo_data = [{'name': 'counties',
             'url': county_topo,
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}]
        geo_file = local_path + 'counties/us_counties.topo.json'
        
        #load geo IDs from local shape file
        with open(geo_file, 'r') as f:
            get_id = json.load(f)
            
        #Grab the FIPS codes from the topo json file and load them into a dataframe
        geometries = get_id['objects']['us_counties.geo']['geometries']
        county_codes = [x['properties']['FIPS'] for x in geometries]
        county_fips_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
        
        #merge geo data with viz data
        merged = pd.merge(data, county_fips_df, on='FIPS', how='outer')
        merged.fillna(0, inplace=True)
        merged.lar_count = merged.lar_count.apply(lambda x: int(x))
        
        #set vis object
        vis = Map(data=merged, geo_data=geo_data, scale=1100, projection='albersUsa',
          data_bind='lar_count', data_key='FIPS',
          map_key={'counties': 'properties.FIPS'})
        
        vis.marks[0].properties.enter.stroke_opacity = ValueRef(value=0.5)
        vis.rebind(column='lar_count', brew='YlGnBu') #set color pattern
        del vis.marks[1].properties.update #shows counties through state color fill
        vis.display() #shows vis object
        return vis
    
    elif geo_level == "state":
        geo_data = [{'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}]
        geo_file = local_path + 'states/usa_states.topo.json'
        #load geo IDs from local shape file
        with open(geo_file, 'r') as f:
            get_id = json.load(f)
            


In [6]:

county_data = pd.read_csv("../data/county_counts.txt", sep="|", na_values=[' '], dtype=object)
county_data.columns = ['FIPS', 'lar_count']
county_data['FIPS_Code'] = county_data['FIPS'].astype(str)
vis = draw_map(data=county_data)
vis.display()

In [7]:
type(vis)

vincent.charts.Map